In [285]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import dtale as dt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from xgboost import XGBClassifier
from collections import defaultdict
pd.options.plotting.backend = "plotly"

#  Data summarization, data preprocessing and feature selections 

Let's format the Health Bank data so that the grain is all indicators for a specific country in 2020. We will also include the HDI index (https://worldpopulationreview.com/country-rankings/hdi-by-country) and this will be the label we want to predict.

In [6]:
hnp = pd.read_csv('./extracted/hnp_stats_csv/HNP_StatsData.csv')
hnpc = pd.read_csv('./extracted/hnp_stats_csv/HNP_StatsCountry.csv')
hdi = pd.read_csv('./extracted/hdi_country.csv')

hnpc_keep = hnpc[['Country Code', 'Table Name', 'Long Name', 'Region', 'Currency Unit', 'Income Group', 'System of trade', 'Government Accounting concept', 'Lending category']]

data = []
for country in hnp['Country Name'].unique():
    row = []
    row.append(country)
    for indicator in hnp['Indicator Name'].unique():
        indicator_value =  hnp[(hnp['Country Name'] == country) & (hnp['Indicator Name'] == indicator)]['2020'].values[0]
        row.append(indicator_value)
    data.append(row)
    
columns = ['Country Name', 'Country Code'] + hnp['Indicator Name'].unique().tolist()
df = pd.DataFrame(data, columns=columns)

In [127]:
country_col = []

missing = ['Bahamas',
 'Brunei',
 'DR Congo',
 'Egypt',
 'Gambia',
 'Hong Kong',
 'Iran',
 'Ivory Coast',
 'Kyrgyzstan',
 'Laos',
 'Micronesia',
 'Palestine',
 'Republic of the Congo',
 'Russia',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Slovakia',
 'South Korea',
 'Syria',
 'Venezuela',
 'Yemen']

for c in df['Country Name'].unique():
    
    if c in hdi['country'].unique():
        country_col.append(c)
    else:
        if ('Bahamas' in c):
            country_col.append('Bahamas')
        elif ('Brunei' in c):
            country_col.append('Brunei')
        elif ('Congo, Dem' in c):
            country_col.append('DR Congo')
        elif ('Egypt' in c):
            country_col.append('Egypt')
        elif 'Gambia' in c:
            country_col.append('Gambia')
        elif 'Hong Kong' in c:
            country_col.append('Hong Kong')
        elif 'Iran' in c:
            country_col.append('Iran')
        elif 'Cote d' in c:
            country_col.append('Ivory Coast')
        elif 'Kyrgy' in c:
            country_col.append('Kyrgyzstan')
        elif 'Lao' in c:
            country_col.append('Laos')
        elif 'Micronesia' in c:
            country_col.append('Micronesia')
        elif 'Palestine' in c:
            country_col.append('Palestine')
        elif ('Congo, Rep' in c):
            country_col.append('Republic of the Congo')
        elif 'Russian' in c:
            country_col.append('Russia')
        elif 'St. Kitts' in c:
            country_col.append('Saint Kitts and Nevis')
        elif 'St. Lucia' in c:
            country_col.append('Saint Lucia')
        elif 'St. Vincent' in c:
            country_col.append('Saint Vincent and the Grenadines')
        elif 'Slovak' in c:
            country_col.append('Slovakia')
        elif 'Korea, Rep' in c:
            country_col.append('South Korea')
        elif 'Syrian' in c:
            country_col.append('Syria')
        elif 'Venezuela' in c:
            country_col.append('Venezuela')
        elif 'Yemen' in c:
            country_col.append('Yemen')
        else:
            country_col.append(np.nan)
            
df['Country Merge'] = country_col
df = df.merge(hdi, left_on='Country Merge', right_on='country')
df = df.drop(['Country Merge', 'country', 'pop2022'], axis=1)
df = df.merge(hnpc_keep, left_on='Country Name', right_on='Table Name')
df = df.drop(['Country Code', 'Table Name', 'Long Name', 'Region', 'Currency Unit'], axis = 1)

In [272]:
df = pd.read_csv('training_data.csv').loc[:, 'Country Name':]

## Data summarization

We'll look at:
1. Missing data for each attribute
2. Attribute with float vs. string types (how many columns that will need to be formatted)
3. The importance of each attribute to the HDI

#### Number of missing rows for each attribute

Below, we can see that out of the 463 attributes, about half of them are missing most of their data. Therefore, as a barrier to eliminate attributes, we'll keep attributes with less than 20% missing values.

In [273]:
df.isna().sum().plot.bar(title='Number of missing values for each Attribute')

In [274]:
df = df.loc[:, df.isin([np.nan]).mean() < .2]

Let's look at missing values now, considering we've removed atributes with more than 20% missing values

We still have approximately 175 attributes (excluding our label, hdi and the country name)


In [275]:
df.isna().sum().plot.bar(title='Number of missing values for each Attribute after removal')

#### Float vs. String attributes

Luckily there are only 4 attributes (including the countries) that are string types. Only those columns will have to be encoded.

In [276]:
types = defaultdict(lambda: 0)
for t in df.dtypes:
    types[t] += 1
    
types = pd.DataFrame(pd.Series(types)).reset_index().rename(columns={0:'Label', 'index': 'Type'}).astype(str)

In [277]:
fig = px.pie(types, values='Label', names='Type', title='Float vs. String attributes')
fig.show()

We'll use `Pandas`'s `OneHotEncoder` to convert catogorical attributes into numerical ones. Pandas uses the function `.get_dummies()` for one-hot encoding.

In [278]:
df = pd.get_dummies(df, columns=["Income Group", "System of trade", "Government Accounting concept"], prefix="transformed")

Now that the encoding is done, we can also go ahead and fill in the null values. We'll replace it them with 0.

In [287]:
df.fillna(0, inplace=True)

#### The importance of each attribute to the HDI

In [292]:
X = pd.concat([df.iloc[:, 1:-9],df.iloc[:, -8:]], axis=1)
y = df['hdi']
model = ExtraTreesRegressor()
model.fit(X,y)
# print(model.feature_importances_)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
fig = feat_importances.plot.bar(title="Importance of each Attribute to Human Development Index")
fig.show()

Given the above graph, we can see that most of our features don't correlate to HDI. Therefore, to make sure our features are important, let's use the features that have an importance greater than 0.005. The list of these features is below.

In [293]:
feat_importances[feat_importances > 0.005]

Age dependency ratio, young                                              0.032482
GNI per capita, Atlas method (current US$)                               0.019407
Human capital index (HCI) (scale 0-1)                                    0.033056
Human capital index (HCI), lower bound (scale 0-1)                       0.040632
Human capital index (HCI), upper bound (scale 0-1)                       0.021149
People using at least basic drinking water services (% of population)    0.006390
People using at least basic sanitation services (% of population)        0.019040
Population ages 00-04, female (% of female population)                   0.011519
Population ages 00-04, male (% of male population)                       0.017292
Population ages 0-14 (% of total population)                             0.036322
Population ages 0-14, female (% of female population)                    0.017526
Population ages 0-14, male (% of male population)                        0.020959
Population ages 

In [300]:
df[['Country Name'] + feat_importances[feat_importances > 0.005].keys().tolist() + ['hdi']].set_index('Country Name').to_csv('final_training_data.csv')